# Export to DuckDB

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

import duckdb

### Load iris data

In [2]:
con = duckdb.connect("../test.db")
iris = con.sql("SELECT * FROM iris").df().drop(columns=['id'])
con.close()

iris.head()

,sepal_length,sepal_width,petal_length,petal_width,label
0,-0.900681,1.019004,-1.340227,-1.315444,0
1,-1.143017,-0.131979,-1.340227,-1.315444,0
2,-1.385353,0.328414,-1.397064,-1.315444,0
3,-1.506521,0.098217,-1.283389,-1.315444,0
4,-1.021849,1.249201,-1.340227,-1.315444,0


In [3]:
iris_data = iris.iloc[:, :4]
iris_label = iris.iloc[:, 4]

### Train the model

In [4]:
x = torch.FloatTensor(iris_data.values)
y = torch.LongTensor(iris_label.values)

In [5]:
class NeuralNetworkClassificationModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetworkClassificationModel,self).__init__()
        model_w = 128
        self.input_layer    = nn.Linear(input_dim, model_w)
        self.hidden_layer1  = nn.Linear(model_w,model_w)
        self.hidden_layer2  = nn.Linear(model_w,model_w)
        self.hidden_layer3  = nn.Linear(model_w,model_w)
        self.hidden_layer4  = nn.Linear(model_w,model_w)
        self.hidden_layer5  = nn.Linear(model_w,64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()

    def forward_train(self, x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.relu(self.hidden_layer2(out))
        out =  self.relu(self.hidden_layer3(out))
        out =  self.relu(self.hidden_layer4(out))
        out =  self.relu(self.hidden_layer5(out))
        out =  self.output_layer(out)
        return out
    
    def forward(self,x):
        out =  self.forward_train(x) #.argmax(dim=1).float()
        return out

In [6]:
input_dim  = 4 
output_dim = 3
model = NeuralNetworkClassificationModel(input_dim,output_dim)

In [7]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
def train_network(model,optimizer,criterion,X_train,y_train,num_epochs,train_losses,test_losses):
    for epoch in range(num_epochs):
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        #forward feed
        output_train = model.forward_train(X_train)

        #calculate the loss
        loss_train = criterion(output_train, y_train)

        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()

        train_losses[epoch] = loss_train.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}")

In [9]:
num_epochs = 1000
train_losses = np.zeros(num_epochs)
test_losses  = np.zeros(num_epochs)

In [10]:
train_network(model,optimizer,criterion,x,y,num_epochs,train_losses,test_losses)

Epoch 50/1000, Train Loss: 0.0206
Epoch 100/1000, Train Loss: 0.0000
Epoch 150/1000, Train Loss: 0.0000
Epoch 200/1000, Train Loss: 0.0000
Epoch 250/1000, Train Loss: 0.0000
Epoch 300/1000, Train Loss: 0.0000
Epoch 350/1000, Train Loss: 0.0000
Epoch 400/1000, Train Loss: 0.0000
Epoch 450/1000, Train Loss: 0.0000
Epoch 500/1000, Train Loss: 0.0000
Epoch 550/1000, Train Loss: 0.0000
Epoch 600/1000, Train Loss: 0.0000
Epoch 650/1000, Train Loss: 0.0000
Epoch 700/1000, Train Loss: 0.0000
Epoch 750/1000, Train Loss: 0.0000
Epoch 800/1000, Train Loss: 0.0000
Epoch 850/1000, Train Loss: 0.0000
Epoch 900/1000, Train Loss: 0.0000
Epoch 950/1000, Train Loss: 0.0000
Epoch 1000/1000, Train Loss: 0.0000


### Compile as TorchScript

In [11]:
sm = torch.jit.script(model)

In [12]:
sm.save("iris_d5_w128.pt")